# 1. Imports and Definitions

## Library Downloads

In [1]:
!pip install -q wandb

## Imports

In [2]:
# generic libs
import numpy as np
import json
import glob

# ML-specific libs
import tensorflow as tf
ks = tf.keras
from sklearn.model_selection import train_test_split

# WandB
import wandb
from wandb.keras import WandbCallback as WandbCallback

## mount Google Drive (for iNaturalist 12K dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. Harness Functions

In [3]:
# initialize model from JSON-style config object
def initModel(config):
  # create layer list
  layers = []

  for lconf in config["layers"]:
    ltype = lconf["type"]
    if   ltype=="input":
      layers.append(ks.Input( 
        shape = lconf["size"]
      ) )
    
    elif ltype=="conv":
      layers.append( ks.layers.Conv2D( 
        lconf["filter_num"],
        lconf["filter_size"], 
        strides = lconf["stride"],
        activation = lconf["activation"] 
      ) )
    
    elif ltype=="pool":
      layers.append( ks.layers.MaxPooling2D( 
        pool_size=lconf["size"] 
      ) )
    
    elif ltype=="fc":
      layers.append( ks.layers.Dense( 
        lconf["size"], 
        activation = lconf["activation"]
      ) )
    
    elif ltype=="flatten":
      layers.append( ks.layers.Flatten() )
    
    elif ltype=="dropout":
      layers.append( ks.layers.Dropout(
        lconf["fraction"]
      ) )
    
    elif ltype=="batchnorm":
      if "perform" in lconf:
        if lconf["perform"]==True:
          layers.append( ks.layers.BatchNormalization() )
      else:
        layers.append( ks.layers.BatchNormalization() )
    
    elif ltype=="exec":
      exec("lexec=ks.layers."+lconf["expr"])
      layers.append(locals()["lexec"])

  model = ks.Sequential(layers)

  # compile and return model for given hyperparameters
  exec( "optexec=ks.optimizers." + config["optimizer"]) #* revise based on final storage pattern used
  opt = locals()["optexec"]
  model.compile( optimizer = opt, loss = config["loss"], metrics = ["accuracy"])
  return model

def trainingRun(config, **kwargs):
  model = initModel(config)
  tp = config["trainparams"]

  # set up WandB callback to train function
  callbacks=[]
  
  try:
    if config["wandb"]:
      callbacks.append(
        WandbCallback(
          monitor="val_loss"
        )
      )
    else: print("Warning: not using WandB")
  except KeyError: print("Warning: not using WandB")
  
  # run training loop for given number of epochs
  if all([_ in kwargs for _ in ["xdata","ydata"]]):
    model.fit(kwargs["xdata"], kwargs["ydata"], epochs = tp["epochs"], batch_size = tp["batch_size"], validation_split = tp["val_split"], callbacks = callbacks)
  elif all([_ in kwargs for _ in ["train_data","val_data"]]):
    model.fit(kwargs["train_data"], validation_data = kwargs["val_data"], epochs = tp["epochs"], batch_size = tp["batch_size"], callbacks = callbacks)
  else: raise AssertionError("improper arguments given")

# 3. Dataset Initialization and Reshaping

## MNIST digit dataset

In [ ]:
# import MNIST dataset
((x_train, y_train), (x_test, y_test)) = ks.datasets.mnist.load_data()

# pad with zeros and convert to 32x32 LeNet-5 input
x_train = np.pad(x_train,((0,0),(2,2),(2,2))).reshape((len(x_train),32,32,1))
x_test = np.pad(x_test,((0,0),(2,2),(2,2))).reshape((len(x_test),32,32,1))

# convert to float and normalize
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

# one hot encode target values
y_train = ks.utils.to_categorical(y_train)
y_test = ks.utils.to_categorical(y_test)

## iNaturalist 12K dataset

In [10]:
inaturalist_train_root = "/content/drive/MyDrive/inaturalist_12K/train"

'''labels = [_.split("/")[-1] for _ in glob.glob(inaturalist_root + "/*")]
def oneHotLabels(i,l):
  ohl = tf.one_hot(l,len(labels))
  print(ohl)
  return i, tf.cast(ohl,tf.float32)'''

def augmentImage(image, label):
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  return image, label

def I12kDatasets(config):
  img_dims = (800,800,3)
  numLabels = config["layers"][-1]["size"]

  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    inaturalist_train_root,
    validation_split=config["trainparams"]["val_split"],
    subset="training",
    label_mode="categorical",
    seed=123,
    image_size=img_dims[:2],
    batch_size=config["trainparams"]["batch_size"])

  val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    inaturalist_train_root,
    validation_split=config["trainparams"]["val_split"],
    subset="validation",
    label_mode="categorical",
    seed=123,
    image_size=img_dims[:2],
    batch_size=config["trainparams"]["batch_size"])

  if "dsAugment" in config["trainparams"] and config["trainparams"]["dsAugment"]==True:
    train_ds = train_ds.map(augmentImage,num_parallel_calls=tf.data.AUTOTUNE)
    val_ds   = val_ds.map(augmentImage,num_parallel_calls=tf.data.AUTOTUNE)
    
  train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  val_ds   = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  
  numLabels = None
  return train_ds, val_ds

# 4. Running A Configuration

## Run-helper functions

In [6]:
# create config object from JSON file
def configFromJSON(arg):
  # input checks
  argtype = type(arg).__name__
  config = None
  if   argtype=="str":
    config = json.load(open(arg,'r')) #* add failsafe
  elif argtype=="TextIOWrapper":
    config = json.load(arg)
  elif argtype=="dict":
    config = arg
  return config

## Sample runs

In [ ]:
wandb.init(project="CNN MNIST Test Runs")
run_config = configFromJSON("lenet5.json")
trainingRun(run_config, xdata=x_train, ydata=y_train)

# 5. Sweeps

## Functions

In [5]:
# create config object for WandB sweep (Assignment 2 architecture)
def configForA2WandbSweep():
  wcfg = wandb.config
  return { "layers": [
    {
      "type": "exec",
      "expr": "experimental.preprocessing.Rescaling(1./255)"
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_1_filter_num,
      "filter_size": wcfg.layer_1_filter_size,
      "stride": wcfg.layer_1_stride,
      "activation": wcfg.layer_1_activation
    },
    {
      "type": "batchnorm",
      "perform": wcfg.bn_perform
    },
    {
      "type": "pool",
      "size": wcfg.layer_1_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_2_filter_num,
      "filter_size": wcfg.layer_2_filter_size,
      "stride": wcfg.layer_2_stride,
      "activation": wcfg.layer_2_activation
    },
    {
      "type": "batchnorm",
      "perform": wcfg.bn_perform
    },
    {
      "type": "pool",
      "size": wcfg.layer_2_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_3_filter_num,
      "filter_size": wcfg.layer_3_filter_size,
      "stride": wcfg.layer_3_stride,
      "activation": wcfg.layer_3_activation
    },
    {
      "type": "batchnorm",
      "perform": wcfg.bn_perform
    },
    {
      "type": "pool",
      "size": wcfg.layer_3_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_4_filter_num,
      "filter_size": wcfg.layer_4_filter_size,
      "stride": wcfg.layer_4_stride,
      "activation": wcfg.layer_4_activation
    },
    {
      "type": "batchnorm",
      "perform": wcfg.bn_perform
    },
    {
      "type": "pool",
      "size": wcfg.layer_4_pool_size
    },
    {
      "type": "conv",
      "filter_num": wcfg.layer_5_filter_num,
      "filter_size": wcfg.layer_5_filter_size,
      "stride": wcfg.layer_5_stride,
      "activation": wcfg.layer_5_activation
    },
    {
      "type": "batchnorm",
      "perform": wcfg.bn_perform
    },
    {
      "type": "pool",
      "size": wcfg.layer_5_pool_size
    },
    {
      "type": "flatten"
    },
    {
      "type": "dropout",
      "fraction": wcfg.layer_6_dropout
    },
    {
      "type": "fc",
      "size": wcfg.layer_6_size,
      "activation": wcfg.layer_6_activation
    },
    {
      "type": "dropout",
      "fraction": wcfg.layer_7_dropout
    },
    {
      "type": "fc",
      "size": wcfg.layer_7_size,
      "activation": wcfg.layer_7_activation
    }
    #* add dropout at strategic location
    ],
    "optimizer": wcfg.optimizer,
    "loss": wcfg.loss,
    "trainparams": {
      "epochs": wcfg.epochs,
      "batch_size": wcfg.batch_size,
      "val_split": wcfg.val_split,
    "dsAugment": wcfg.dsAugment
    },
    "wandb": True
  }

def runWandbSweep():
  wandb.init()
  # obtain config and base(i.e. un-augmented) datasets
  run_config = configForA2WandbSweep()
  train_dataset, val_dataset = I12kDatasets(run_config)
  
  # perform training run
  trainingRun(run_config, train_data=train_dataset, val_data=val_dataset)

In [ ]:
r_config = configFromJSON("A2arch.json")
train_dataset, val_dataset = I12kDatasets(r_config)

# perform training run
trainingRun(r_config, train_data=train_dataset, val_data=val_dataset)

## Configure and run sweep

In [ ]:
wandbSweepCfg = {
  "name":"iNaturalist Parameter Sweep", 
  "metric":{
    "name":"val_loss",
    "goal":"minimize"
  }, 
  "method": "bayes", 
  "parameters":{
    # layer parameters
    "layer_1_filter_num": { "values": [8,16,32] },
    "layer_1_filter_size": { "values": [3,7] },
    "layer_1_stride": { "values": [1] },
    "layer_1_activation": { "values": ["relu"] },
    #"layer_1_bn_perform": { "values": [False] },
    "layer_1_pool_size": { "values": [2] },

    "layer_2_filter_num": { "values": [8,16,32] },
    "layer_2_filter_size": { "values": [3,7] },
    "layer_2_stride": { "values": [1] },
    "layer_2_activation": { "values": ["relu"] },
    #"layer_2_bn_perform": { "values": [False] },
    "layer_2_pool_size": { "values": [2] },

    "layer_3_filter_num": { "values": [8,16,32] },
    "layer_3_filter_size": { "values": [3,7] },
    "layer_3_stride": { "values": [1] },
    "layer_3_activation": { "values": ["relu"] },
    #"layer_3_bn_perform": { "values": [False] },
    "layer_3_pool_size": { "values": [2] },

    "layer_4_filter_num": { "values": [8,16,32] },
    "layer_4_filter_size": { "values": [3,7] },
    "layer_4_stride": { "values": [1] },
    "layer_4_activation": { "values": ["relu"] },
    #"layer_4_bn_perform": { "values": [False] },
    "layer_4_pool_size": { "values": [2] },

    "layer_5_filter_num": { "values": [8,16,32] },
    "layer_5_filter_size": { "values": [3,7] },
    "layer_5_stride": { "values": [1] },
    "layer_5_activation": { "values": ["relu"] },
    #"layer_5_bn_perform": { "values": [False] },
    "layer_5_pool_size": { "values": [2] },
    
    "layer_6_dropout": { "values": [0.2,0.3] },
    "layer_6_size": { "values": [64, 96, 128] },
    "layer_6_activation": { "values": ["tanh"] },

    "layer_7_dropout": { "values": [0,0.2,0.3] },
    "layer_7_size": { "values": [10] },
    "layer_7_activation": { "values": ["softmax"] },

    "bn_perform": { "values": [False, True] },

    # optimizer and loss
    "optimizer": { "values": ["Adam()"] },
    "loss": { "values": ["categorical_crossentropy"] },

    # training parameters
    "epochs": { "values":[5] },
    "batch_size": { "values":[32] },
    "val_split": { "values": [0.1] },
    "dsAugment": { "values": [False, True]}
  }
}

sweepId = wandb.sweep(wandbSweepCfg)
wandb.agent(sweepId, function = runWandbSweep)